In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [51]:
from models.base import Base
base = Base(16, 128)
samples, base_log_det = base.sample_and_log_prob(128)
samples.shape


torch.Size([128, 10])

## models.base.Base & models.bijectors.CircularShift Test

Problem: nflow generate samples whose require_grad=False

In [52]:
from models.bijectors import CircularShift
from torch.nn import Parameter
samples.requires_grad=True
shift = Parameter(
    torch.rand(size=(1, )))
circular_shift = CircularShift(shift, -torch.pi, torch.pi)
shift_result, shift_logdet = circular_shift(samples)
shift_mean = shift_result.sum()
shift_mean.backward()
print(samples.grad)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


In [89]:
from models.model import make_model
from experiments.configs import get_config
config = get_config(16)
model, energy_fn = make_model(
    -torch.pi, torch.pi, **config.model['kwargs'])


In [90]:
from models.transforms import Dihedral2Coord
from models.energy import Energy
torch.autograd.set_detect_anomaly(True)
trans = Dihedral2Coord(
    mol=model._distribution.mol,
    angles=model._distribution.torsion_angles)
energy = Energy(
    mol=model._distribution.mol)

In [91]:
model._distribution.torsion_angles

tensor([[11,  0,  1,  2],
        [10,  1,  2,  3],
        [ 6,  2,  3,  5],
        [15,  4,  0, 11],
        [14,  5,  3,  2],
        [ 7,  6,  2,  3],
        [ 8,  7,  6,  2],
        [ 9,  8,  7,  6],
        [12,  9,  8,  7]])

In [94]:
angles_sample = torch.rand(size=(128, 9), requires_grad=True)

In [115]:
coord_sample = trans(angles_sample)
# energy_sample = energy(coord_sample)
loss = coord_sample.sum()
loss.backward()


tensor(1.0490e-05, grad_fn=<MaxBackward1>) tensor(13194) tensor(12103)


c:\Users\继续革命\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Error detected in BmmBackward0. Traceback of forward call that caused the error:
  File "c:\Users\继续革命\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\继续革命\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\继续革命\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\Users\继续革命\AppData\Local\Programs\Python\Python38\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "c:\Users\继续革命\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "c:\Users\继续革命\AppData\Local\Programs\Python\Python38\

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [128, 3, 1]], which is output 0 of AsStridedBackward0, is at version 1515; expected version 1512 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [32]:
print(linear_layer.weight.grad)

None


In [17]:
loss, logdet = model.sample_and_log_prob(128)


In [22]:
real_loss = (loss + logdet[:, None]).mean()


In [24]:
real_loss.backward()

In [48]:
model._transform._transforms[0]._transforms[1].conditioner.linear1.weight.grad


tensor([[ 0.0239, -0.3383,  0.3175,  ..., -0.1333, -0.3284, -0.4040],
        [-0.1827,  1.1365, -0.5813,  ..., -0.3947,  0.3445,  0.7744],
        [-0.0460,  0.5030, -0.5148,  ..., -0.1440,  0.2224,  0.9553],
        ...,
        [ 0.9095, -0.1586,  0.2059,  ...,  0.0287, -0.6542, -0.1489],
        [-0.2702, -0.0838,  0.3339,  ..., -0.3490,  0.1917, -0.4530],
        [-0.1773,  0.2773, -0.5189,  ..., -0.1973, -0.0033,  0.5414]])

: 

In [12]:
j = torch.rand(size=(4, 17))
k = torch.rand(size=(4, 17))
l = torch.rand(size=(4, 17))
ls = [j, k, l]
torch.stack(ls).shape

torch.Size([3, 4, 17])

In [32]:
import torch
i=torch.rand(size=(5,4))
j=torch.rand(size=(4, 17))
k=torch.rand(size=(4, 17))
l=torch.rand(size=(4, 17))
from models.spline import _rational_quadratic_spline_fwd
_rational_quadratic_spline_fwd(i,j,k,l)

(tensor([[0.8167, 0.5797, 3.3655, 0.3703],
         [0.7785, 2.3855, 0.4710, 2.0032],
         [0.9357, 2.4022, 0.5809, 1.4372],
         [0.9357, 2.4022, 0.4772, 0.6444],
         [0.8134, 0.5167, 0.5041, 1.9168]]),
 tensor([[-2.0675, -1.5534,     nan, -0.5470],
         [-2.0675,     nan, -1.2066,     nan],
         [    nan,     nan, -0.1418,     nan],
         [    nan,     nan, -1.2066, -1.1884],
         [-2.0675, -1.5534, -0.1418,     nan]]))

In [39]:
from torch import Tensor
from typing import Tuple
def _rqs_fwd_single(x: Tensor,
                                   x_pos: Tensor,
                                   y_pos: Tensor,
                                   knot_slopes: Tensor) -> Tuple[Tensor, Tensor]:
  """Applies a rational-quadratic spline to a scalar.
  Args:
    x: a scalar (0-dimensional array). The scalar `x` can be any real number; it
      will be transformed by the spline if it's in the closed interval
      `[x_pos[0], x_pos[-1]]`, and it will be transformed linearly if it's
      outside that interval.
    x_pos: array of shape [num_bins + 1], the bin boundaries on the x axis.
    y_pos: array of shape [num_bins + 1], the bin boundaries on the y axis.
    knot_slopes: array of shape [num_bins + 1], the slopes at the knot points.
  Returns:
    A tuple of two scalars: the output of the transformation and the log of the
    absolute first derivative at `x`.
  """
  # Search to find the right bin. NOTE: The bins are sorted, so we could use
  # binary search, but this is more GPU/TPU friendly.
  # The following implementation avoids indexing for faster TPU computation.
  below_range = x <= x_pos[0]
  above_range = x >= x_pos[-1]
  correct_bin = torch.logical_and(x >= x_pos[:-1], x < x_pos[1:])
  any_bin_in_range = torch.any(correct_bin)
  first_bin = torch.concat([torch.tensor([1], dtype=bool),
                               torch.zeros(len(correct_bin)-1, dtype=bool)])
  # If y does not fall into any bin, we use the first spline in the following
  # computations to avoid numerical issues.
  correct_bin = torch.where(any_bin_in_range, correct_bin, first_bin)
  # Dot product of each parameter with the correct bin mask.
  params = torch.stack([x_pos, y_pos, knot_slopes], axis=1)
  params_bin_left = torch.sum(correct_bin[:, None] * params[:-1], axis=0)
  params_bin_right = torch.sum(correct_bin[:, None] * params[1:], axis=0)

  x_pos_bin = (params_bin_left[0], params_bin_right[0])
  y_pos_bin = (params_bin_left[1], params_bin_right[1])
  knot_slopes_bin = (params_bin_left[2], params_bin_right[2])

  bin_width = x_pos_bin[1] - x_pos_bin[0]
  bin_height = y_pos_bin[1] - y_pos_bin[0]
  bin_slope = bin_height / bin_width

  z = (x - x_pos_bin[0]) / bin_width
  # `z` should be in range [0, 1] to avoid NaNs later. This can happen because
  # of small floating point issues or when x is outside of the range of bins.
  # To avoid all problems, we restrict z in [0, 1].
  z = torch.clip(z, 0., 1.)
  sq_z = z * z
  z1mz = z - sq_z  # z(1-z)
  sq_1mz = (1. - z) ** 2
  slopes_term = knot_slopes_bin[1] + knot_slopes_bin[0] - 2. * bin_slope
  numerator = bin_height * (bin_slope * sq_z + knot_slopes_bin[0] * z1mz)
  denominator = bin_slope + slopes_term * z1mz
  y = y_pos_bin[0] + numerator / denominator

  # Compute log det Jacobian.
  # The logdet is a sum of 3 logs. It is easy to see that the inputs of the
  # first two logs are guaranteed to be positive because we ensured that z is in
  # [0, 1]. This is also true of the log(denominator) because:
  # denominator
  # == bin_slope + (knot_slopes_bin[1] + knot_slopes_bin[0] - 2 * bin_slope) *
  # z*(1-z)
  # >= bin_slope - 2 * bin_slope * z * (1-z)
  # >= bin_slope - 2 * bin_slope * (1/4)
  # == bin_slope / 2
  logdet = 2. * torch.log(bin_slope) + torch.log(
      knot_slopes_bin[1] * sq_z + 2. * bin_slope * z1mz +
      knot_slopes_bin[0] * sq_1mz) - 2. * torch.log(denominator)

  # If x is outside the spline range, we default to a linear transformation.
  y = torch.where(below_range, (x - x_pos[0]) * knot_slopes[0] + y_pos[0], y)
  y = torch.where(above_range, (x - x_pos[-1]) * knot_slopes[-1] + y_pos[-1], y)
  logdet = torch.where(below_range, torch.log(knot_slopes[0]), logdet)
  logdet = torch.where(above_range, torch.log(knot_slopes[-1]), logdet)
  return y, logdet
